# Week 5- integration and ODEs with numpy and scipy

Integration is something that we have to do all the time in scientific computing. There are generally two situations that we are faced with:
* we have the value of the function only at certain values of its parameters, for example because it comes from measurements or is very time-consuming to evaluate
* we are integrating a known function that we can evaluate for any value of its parameters that we want

You've seen the first situation before in calculus classes, with concepts like the trapezoidal rule and Simpson's rule. You already saw `numpy.trapz` in Lab 2, 
so let's look at it in a bit more detail and at some of the other options.

`help(numpy.trapz)` shows us how trapz works. It actually doesn't have many parameters, just integrates according to the trapezoidal rule and it's pretty simple to use.

## Integration

In [ ]:
import numpy as np
import scipy.integrate as scint
import matplotlib.pyplot as plt

In [ ]:
# not very accurate
x = np.arange(0,10)
y = x**2
int_result = np.trapz(y,x)
print(int_result, 10**3/3 )

In [ ]:
# more steps helps
x = np.linspace(0,10,100)
y = x**2
int_result = np.trapz(y,x)
print(int_result, 10**3/3 )

There are lots more options for integrating with scipy.integrate, type `help(scint)` to see the list. Let's see how `scipy.integrate.romb` works.

In [ ]:
int_result = scint.romb(y, x)
print(int_result, 10**3/3 )

In [ ]:
# more accurate than trapezoidal rule even with fewer sample points
# note different calling structure
x = np.linspace(0,10,65)
y = x**2
dx = x[1]-x[0]
int_result = scint.romb(y, dx = dx)
print(int_result, 10**3/3 )

OK, so now let's try some of the `scipy.integrate` routines where instead of having only the value of the function, we can evaluate the function itself. Here we need to pass the function as an argument to the `scipy` routine. The tex talks about this on page 62-64. It's usually not complicated but you do have to read the documentation carefully to understand what the routine is expecting.

In [ ]:
help(scint.quad)

In [ ]:
# let's define a function
def myfunc(x):
    return(x**2 + np.sin(x))

In [ ]:
result = scint.quad(myfunc,1,4)

In [ ]:
# the resulting definite integral
result

In [ ]:
# let's look at more detail - read the documentation for scint.quad to see what all this stuff is
result = scint.quad(myfunc,1,4, full_output=1)
print(result)

In [ ]:
# what if we want our function to have parameters?
def myfunc2(x, a=1):
    return(x**2 + a*np.sin(x))

In [ ]:
result = scint.quad(myfunc2,1,4)
print(result)

In [ ]:
result = scint.quad(myfunc2,1,4, args= (0,))
print(result)

`scint.quad` is the general-purpose integrator and there are variations for double and triple integrals as well as vector-valued functions. You can also use it to do indefinite integrals:

In [ ]:
def myfunc3(x): 
    return(3*np.exp(-x))

scint.quad(myfunc3, 0, np.inf)
(1.0, 5.842605999138044e-11)

In [ ]:
# but of course they have to converge 
def myfunc3(x): 
    return(3*np.exp(x))

result = scint.quad(myfunc3, 0, np.inf)
print(result)

These routines return an estimate of the uncertainty in the reuslt and you want to look at that carefully. One of the nice things about `numpy` and `scipy` is that they are open-source, so you can look at the source code - which is linked from the documentation - and see exactly what it's doing. [Here](https://github.com/scipy/scipy/blob/v1.5.2/scipy/integrate/quadpack.py#L49-L442) it is for `scipy.quad`.

## Solving ODEs

There are also routines in `scipy.integrate` that solve odinary differential equations. Something important to note is that the SciPy lectures document dsecribes an older version of the code where `odeint` was the major routine. The new version is `solve_ivp`, so let's see how that works with the examples in SciPy Lectures.

In [ ]:
help(scint.solve_ivp)

We want to solve the ODE dy/dt = -2 y between t = 0 and 4, with the initial condition y(t=0) = 1. First we define the function giving dy/dt"

In [ ]:
def dydt(t, y):
    return(-2*y)

In [ ]:
# now we call solve_ivp:
result = scint.solve_ivp(dydt, (0,4), np.array([1.0]))
print(result)

In [ ]:
#%matplotlib inline
fig, ax = plt.subplots()
#ax.plot(result.t, result.y)
ax.plot(result.t, result.y[0])

The default behaviour gives us a curve that's not very smooth, since there aren't that many points. What if we do:

In [ ]:
t_soln = np.linspace(0,4,100)
result = scint.solve_ivp(dydt, (0,4), np.array([1.0]),t_eval= t_soln )
print(result)

In [ ]:
# that looks better
fig, ax = plt.subplots()
ax.plot(result.t, result.y[0])

OK let's try a more complicated ODE: a damped spring-mass oscillator. The position of a mass attached to a spring obeys the 2nd order ODE $y'' + 2 \varepsilon \omega_0 y' + \omega_0^2 y = 0$ with $\omega_0^2 = k/m$ with $k$ the spring constant, $m$ the mass and $\varepsilon = c/(2 m \omega_0)$ with $c$ the damping coefficient.

In [ ]:
# define some variables and compute some constants
mass = 0.5  # kg
kspring = 4  # N/m
cviscous = 0.4  # N s/m
eps = cviscous / (2 * mass * np.sqrt(kspring/mass))
omega = np.sqrt(kspring / mass)

The tricky bit here is to consider our derivative to be the derivative of the **state vector** $[y, y']$. Then we write our derivative function so it returns the separate derivatives of these two things:
$d/dt[y, y'] = [y', y''] = [y', - 2 \varepsilon \omega_0 y' - \omega_0^2 y]$

In [ ]:
# note that neither derivative is an explicit function of time 
# so the time parameter doesn't do anything here but has to be included since it's required by solve_ivp
def deriv_ystate(time, ystate, eps, omega):
    return(ystate[1], -2.0 * eps * omega * ystate[1] - omega **2 * ystate[0])

In [ ]:
# note the way more function evaluations here (1046 compared to 62  in the previous example)
time_soln = np.linspace(0, 10, 100)
yinit = (1, 0)
result = scint.solve_ivp(deriv_ystate, (0,100), yinit, t_eval=time_soln, args = (eps, omega,) )

In [ ]:
fig, ax = plt.subplots()
ax.plot(result.t, result.y[0])